In [ ]:
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [4]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


## load in data

In [145]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [146]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [147]:
findfirst(x -> x[:subject_id] == "6.100A", all_classes)

3370

In [157]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_inds_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2962, 3303, 3304]
 [3856, 3857, 3858, 3859]
 [3860, 3861, 3862]
 [1536, 1537]
 [1538, 1540, 1539]
 [3752, 3753, 3754, 3755, 3756]

In [175]:
ai_programming_ids = ["6.100A", "6.100L"]
ai_math_1_ids = ["6.1200"]
ai_math_2_ids = ["6.S084", "18.C06", "18.06"]
ai_math_3_ids = ["6.3700", "6.3800", "18.05"]
ai_foundation_ids = ["6.1010", "6.1210"]
ai_centers_ids = ["6.1220", "6.1400", "6.3000", "6.3100", "6.3260", "6.3720", "6.3900", "6.3950",
                "6.4110", "6.4120", "6.4400", "6.4590", "6.7201", "6.C35", "9.660"] # removed "6.C571" and "6.C01"

ai_data_center_ids = ["6.3720", "6.3900"] # no "6.C01"
ai_model_center_ids = ["6.3000", "6.3100", "6.4110", "6.4400"]
ai_dec_center_ids = ["6.3100", "6.4110", "6.7201"]  # no "6.C571"
ai_comp_center_ids = ["6.1220", "6.1400", "6.4400", "6.7201"] # no "6.C571"
ai_human_center_ids = ["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "9.660"]
ai_centers_ids_list = [ai_data_center_ids, ai_model_center_ids, ai_dec_center_ids, ai_comp_center_ids, ai_human_center_ids]
ai_centers_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_center] for ai_center in ai_centers_ids_list]

ai_ids = [ai_programming_ids, ai_math_1_ids, ai_math_2_ids, ai_math_3_ids, ai_foundation_ids, ai_centers_ids]  # DOESN'T INCLUDE AI_MATH_IDS YET
ai_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_ids]

6-element Vector{Vector{Int64}}:
 [3370, 3372]
 [3379]
 [3693, 1701, 1546]
 [3437, 3443, 1545]
 [3373, 3381]
 [3382, 3384, 3427, 3429, 3431, 3439, 3444, 3445, 3449, 3450, 3459, 3466, 3550, 3669, 4025]

In [150]:
all_classes[3379]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 30 entries:
  :rating             => 5.03
  :gir_attribute      => "REST"
  :has_final          => true
  :description        => "Elementary discrete mathematics for science and engin…
  :offered_fall       => true
  :offered_spring     => true
  :instructors        => ["F. Leighton"]
  :preparation_units  => 7
  :joint_subjects     => ["18.062"]
  :out_of_class_hours => 6.49
  :total_units        => 12
  :related_subjects   => ["6.120A", "6.5340", "6.7480", "6.7240", "6.8410", "6.…
  :pdf_option         => false
  :in_class_hours     => 4.64
  :is_half_class      => false
  :schedule           => "Lecture,26-100/TR/0/2.30-4;Recitation,26-168/WF/0/10,…
  :level              => "U"
  :prerequisites      => "GIR:CAL1"
  :subject_id         => "6.1200"
  :title              => "Mathematics for Computer Science"
  :lab_units          => 0
  :design_units       => 0
  :public

## optimize

In [179]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)
@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

##
# CONSTRAINTS
##

# full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);

# wellbeing: no more than 112 hours per week
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);

# not repeatable: cannot take a class again
@constraint(model, [i in 1:n], sum(X[i, t] for t=1:S) <= 1);

# GIR constraints
@constraint(model, [gir_type in gir_inds_types], sum(X[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1);

# taken correct semester
@constraint(model, [s in 1:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_fall]);
@constraint(model, [s in 2:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_spring]);

# 2 CI-H or HW
@constraint(model, sum(
    (get(all_classes[i], :communication_requirement, "") in ["CI-H", "CI-HW"]) * X[i, s] for i=1:n, s=1:S) >= 2);

# 8 HASS
@constraint(model, sum(
    (get(all_classes[i], :hass_attribute, "") in ["HASS-H", "HASS-A", "HASS-S"]) * X[i, s] for i=1:n, s=1:S) >= 8);

# 1 of each HASS-H, A, S
@constraint(model, [hass_type in ["HASS-H", "HASS-A", "HASS-S"]], sum(
    (get(all_classes[i], :hass_attribute, "") in [hass_type]) * X[i, s] for i=1:n, s=1:S) >= 1);


hard-coding in the 6-4 requirements

In [180]:
# ai_inds_types, 6 elem vector

# 1 programming skills
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_inds_types[1], s in 1:S) == 1);

# 3 math
ai_math_types = [ai_inds_types[2], ai_inds_types[3], ai_inds_types[4]];
@constraint(model, [math_type in ai_math_types], sum(X[math_ind, s] for math_ind in math_type, s in 1:S) == 1);

# 2 foundation
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_inds_types[5], s in 1:S) == 2);

# 5 total center
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_inds_types[6], s in 1:S) >= 5);

# 1 per center
@constraint(model, [ai_center in ai_centers_inds_types], sum(X[ai_ind, s] for ai_ind in ai_center, s in 1:S) >= 1);

# 4 elective

In [181]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42365 rows, 37624 columns and 164344 nonzeros
Model fingerprint: 0x1a2a41fc
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 179.4500000
Presolve removed 38275 rows and 19820 columns
Presolve time: 0.24s
Presolved: 4090 rows, 17804 columns, 63188 nonzeros
Found heuristic solution: objective 160.2540000
Variable types: 0 continuous, 17804 integer (17804 binary)
Found heuristic solution: objective 153.7960000

Root relaxation: objective 1.283960e+02, 84 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds  

In [182]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [1180, 2021, 3373, 3381, 3429]
 [816, 1546, 3384]
 [1539, 2600, 3861]
 [2962, 3444, 3857]
 [294, 296, 2625, 2885, 3017, 4141]
 [1536, 2573, 2574, 3372, 3431]
 [2661, 3445, 3755]
 [1545, 3379, 4295]

## Interpreter

In [170]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

function interpret_course_attr(classes_id_per_sem, attr)
    courses_attr = [[get(all_classes[class_ind], Symbol(attr), "") for class_ind in sem] for sem in classes_id_per_sem]
    return courses_attr
end

function interpret_course_hours(classes_id_per_sem)
    course_hours = [[H[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_hours
end

function interpret_course_units(classes_id_per_sem)
    course_units = [[U[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_units
end

interpret_course_units (generic function with 1 method)

In [183]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["15.839", "20.S900", "6.1010", "6.1210", "6.3100"]
 ["14.399", "18.06", "6.1400"]
 ["18.022", "21M.622", "8.021"]
 ["3.091", "6.3900", "8.011"]
 ["10.960", "10.991", "21M.747", "24.93", "3.903", "CMS.627"]
 ["18.01", "21M.451", "21M.460", "6.100L", "6.3260"]
 ["21W.032", "6.3950", "7.015"]
 ["18.05", "6.1200", "ES.113"]

In [171]:
interpret_course_attr(taken_classes_sem, "title")

8-element Vector{Vector{String}}:
 ["Seminar in Polymers and Soft Matter", "Workshop in Marketing", "Special Subject in Biological Engineering", "The Search for Meaning", "Seminar in Polymers and Soft Matter", "Introduction to Computer Science and Programming", "Fundamentals of Programming", "Physics II"]
 ["Land-Atmosphere Interactions", "Introduction to Probability and Statistics", "Physics I"]
 ["Physical Improvisation: Scores and Structures", "Science Writing and New Media: Introduction to Digital Media", "Introductory Biology"]
 ["Calculus", "Introduction to Solid-State Chemistry", "Introduction to Algorithms"]
 ["Calculus", "Collaborative Piano", "AI, Decision Making, and Society", "Imagination, Computation, and Expression Studio"]
 ["Linear Algebra", "Networks", "Ancient Greek Philosophy and Mathematics"]
 ["Computational Cognitive Science", "Foundations of Information Policy", "Computational Cognitive Science"]
 ["Seminar in Data Economics and Development Policy", "MIT Senegale

In [168]:
sems_hours = interpret_course_hours(taken_classes_sem)
[sum(sem_hours) for sem_hours in sems_hours]

8-element Vector{Float64}:
 38.53999999999999
 23.21
 18.43
 28.14
 24.25
 21.72
 28.78
 19.09

In [167]:
sems_units = interpret_course_units(taken_classes_sem)

8-element Vector{Vector{Int64}}:
 [2, 0, 0, 2, 2, 6, 12, 12]
 [12, 12, 12]
 [12, 12, 12]
 [12, 12, 12]
 [12, 0, 12, 12]
 [12, 12, 12]
 [12, 12, 12]
 [12, 6, 12, 6]